## Infos

Da es sich um ein lokales Docker Projekt handelt, muss Docker Desktop installiert sein.
Für dieses Notebook kann die requirements.txt mit python 3.13.3 genutzt werden.
Nach dem starten der docker-compose kann es losgehen, sobald alle container gebaut wurden :)

## Datenbeschaffung

Zunächst muss die Datenbank generiert werden. Es werden ca 500.000 Datenpunkte aus dem Archiv von OpenSense geladen. Die laufzeit beträgt ca 15. minuten

In [60]:
import requests
response = requests.get("http://localhost:8000/create-db")
response.json()

{'status': 'created',
 'info': {'from': '2022-07-25 02:01:49',
  'to': '2025-06-29 23:34:51',
  'total_measurements': 519067}}

## Modell training

Das Modelltraining muss einmal über dieses Notebook gestartet werden. Dafür können wir jetzt die Daten von der Datenbank abfragen.

In [61]:
import requests
import pandas as pd
response = requests.get("http://localhost:8000/get-db")
df = pd.DataFrame(response.json()["data"])
df['createdat'] = pd.to_datetime(df['createdat'], format="ISO8601", utc=True)
df

,createdat,pm10,pm25,temperatur,rel_luftfeuchte,luftdruck
0,2022-07-25 00:01:49.463000+00:00,10.73,9.93,22.98,73.16,99784.28
1,2022-07-25 00:04:39.011000+00:00,10.80,9.80,23.02,71.94,99785.34
2,2022-07-25 00:07:27.816000+00:00,11.70,10.70,22.95,71.77,99781.03
3,2022-07-25 00:10:16.348000+00:00,11.77,10.77,22.95,71.67,99777.47
4,2022-07-25 00:13:04.992000+00:00,11.10,10.20,22.89,72.10,99773.19
...,...,...,...,...,...,...
519062,2025-06-29 21:23:28.881000+00:00,4.03,3.73,25.45,70.21,100483.41
519063,2025-06-29 21:26:16.735000+00:00,3.37,3.07,25.39,71.26,100484.56
519064,2025-06-29 21:29:10.554000+00:00,3.57,3.27,25.23,72.35,100488.56
519065,2025-06-29 21:31:58.780000+00:00,3.47,3.10,24.68,72.74,100420.53


In [64]:
df.dtypes

createdat          datetime64[ns, UTC]
pm10                            object
pm25                            object
temperatur                      object
rel_luftfeuchte                 object
luftdruck                       object
dtype: object

Jetzt kann das Modell trainiert werden.

In [66]:
import numpy as np
df = df.replace([np.nan, np.inf, -np.inf], None)
# Vorbereitung für JSON: Zeitstempel als Strings
payload = df[["createdat", "temperatur"]].copy()
payload["createdat"] = payload["createdat"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

json_payload = payload.to_dict()

response = requests.post("http://localhost:9000/train", json=json_payload)
response.json()

{'message': 'Model trained and saved.'}

Jetzt kann man das dashboard frei benutzen. Die Datenbank füllt sich automatisch mit den neuen Daten, sobald das Dashboard geöffnet wird.